In [24]:
import os
import pandas as pd
import sqlite3 as sqlite

In [25]:
data = pd.read_csv("scraped.csv").query("team != 'team/'")

In [26]:
names = pd.read_csv("names.csv", encoding="utf-8")

In [27]:
sport_ents = names.copy()

In [28]:
graph = pd.DataFrame()
graph["A"] = data["rider"]
graph["B"] = -data["team"]
graph = graph[graph.isin(set(sport_ents["url"])).all(axis=1)]

graph_sql = graph.to_numpy().tolist() + graph.to_numpy()[:,::-1].tolist()

In [39]:
path_to_db = "../docs/graph.sqlite"
try:
    try:
        os.remove(path_to_db)
    except:
        conn.close()
        del conn
        os.remove(path_to_db)
except FileNotFoundError:
    pass

In [40]:
bad_team_rider_combos = [(18468, -17275), (2665, -2305), (2665, -1768)]
bad_team_rider_combos = " or ".join(f"(a={i} and b ={j}) or (a={j} and b={i})" for i,j in bad_team_rider_combos)

with sqlite.connect(path_to_db) as conn:
    conn.execute("""CREATE TABLE sport_ents (
                    name text NOT NULL,
                    url int NOT NULL,
                    year int NOT NULL
                ) """)
    conn.execute("CREATE INDEX urls ON sport_ents (url) ")
    conn.executemany("INSERT INTO sport_ents VALUES (?, ?, ?)", sport_ents.to_numpy().tolist())
    conn.execute("""CREATE TABLE graph (
                    a int NOT NULL,
                    b int NOT NULL)""")
    conn.executemany("INSERT INTO graph VALUES (?, ?)", graph_sql)
    # remove bad entries
    conn.execute(f"delete from graph where {bad_team_rider_combos}")
    conn.execute("CREATE INDEX graph_roots ON graph (a) ")
conn.close()
del conn

'(a=18468 and b =-17275) or a=-17275 and b=18468) or (a=2665 and b =-2305) or a=-2305 and b=2665) or (a=2665 and b =-1768) or a=-1768 and b=2665)'